In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import generator

#config = tf.ConfigProto(device_count = {'DML': 0})
# sess = tf.Session(config=config)

In [2]:
def generate_dataset():
    gen = generator.Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
                        path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
                        path_p_d="data/initial kernels/Pressure_Carbon.npy",
                        path_p_s="data/initial kernels/Pressure_Carbon.npy",
                        path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
                )
    gen.generate_data_set()

def show_dataset():
        dataset = hkl.load('data/datasets/carbon3.hkl')
        i = 8
        plt.plot(dataset[i]["isotherm"], marker=".")
        plt.plot(dataset[i]["pore_distribution"], marker=".")
        plt.show()
        print(len(dataset))

In [3]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)
    #plt.yscale("log")
    plt.show()

In [30]:
def load_dataset(path, type=0):
    min_exp_pressure_i = 40
    max_exp_pressure_i = 458
    with open(path, 'rb') as f:
            dataset = np.load(f)
            isotherm_data = dataset["isotherm_data"]
            pore_distribution_data = dataset["pore_distribution_data"]
    x = np.empty((isotherm_data.shape[0], (-min_exp_pressure_i + max_exp_pressure_i)))
    #y = np.empty(pore_distribution_data.shape)
    y = np.empty(shape=(len(isotherm_data), 2))
    for i in range(len(isotherm_data)):
        isotherm = isotherm_data[i][min_exp_pressure_i:max_exp_pressure_i]
        isotherm -= min(isotherm)
        isotherm /= max(isotherm)
        pore_distribution = pore_distribution_data[i] - min(pore_distribution_data[i])
        pore_distribution /= max(pore_distribution)
        x[i] = isotherm
        #y[i] = pore_distribution
        if type == 0:
            y[i] = np.array([0, 1])
        else:
            y[i] = np.array([1, 0])
    x, y = shuffle(x, y)
    return x, y

x1, y1 = load_dataset('data/datasets/Silica_classification.npz', 0) # silica
x2, y2 = load_dataset('data/datasets/Carbon_classification.npz', 1) # carbon
x = np.concatenate((x1, x2))
y = np.concatenate((y1, y2))

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [38]:
## DENSE NET
# model = Sequential(
#     [
#         Input(shape=len(x_train[0], )),
#         layers.Dense(400, activation='relu'),
#         layers.Dropout(0.2),
#         layers.Dense(400, activation='relu'),
#         layers.Dropout(0.2),
#         layers.Dense(400, activation='relu'),
#         layers.Dropout(0.2),
#         layers.Dense(400, activation='relu'),
#         layers.Dropout(0.2),
#         layers.Dense(400, activation='relu'),
#         layers.Dropout(0.2),
#         layers.Dense(400, activation='relu'),
#         layers.Dropout(0.2),
#         layers.Dense(400, activation='relu'),
#         layers.Dropout(0.2),
#         layers.Dense(len(y_train[0]), activation='relu') #  activation=tf.math.abs
#     ]
# )
### CONV NET 
model = Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(len(x_train[0]),1)))
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [39]:
#model.compile(loss='mean_squared_error', optimizer='Adam')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
# mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/classification.keras', save_best_only=True,
#                                            monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
#                                            save_freq='epoch')
mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/classification.keras', save_best_only=True,
                                           monitor='accuracy', mode='max', verbose=1, save_weights_only=False,
                                           save_freq='epoch')

reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                   patience=100, verbose=1, mode='auto')
# from keras import backend as K
# K.set_value(model.optimizer.learning_rate, 0.001)
history = model.fit(np.array(x_train), np.array(y_train),
                    epochs=50, batch_size=5000, shuffle=True,
                    validation_data=(np.array(x_test), np.array(y_test)), callbacks=[mcp_save])

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.7055 - accuracy: 0.5389
Epoch 1: accuracy improved from -inf to 0.53892, saving model to data/models\classification.keras
10/10 [==============================] - 5s 464ms/step - loss: 0.7055 - accuracy: 0.5389 - val_loss: 0.6738 - val_accuracy: 0.5925
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.6569 - accuracy: 0.6075
Epoch 2: accuracy improved from 0.53892 to 0.60753, saving model to data/models\classification.keras
10/10 [==============================] - 4s 445ms/step - loss: 0.6569 - accuracy: 0.6075 - val_loss: 0.6459 - val_accuracy: 0.6259
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.6191 - accuracy: 0.6435
Epoch 3: accuracy improved from 0.60753 to 0.64354, saving model to data/models\classification.keras
10/10 [==============================] - 4s 446ms/step - loss: 0.6191 - accuracy: 0.6435 - val_loss: 0.6071 - val_accuracy: 0.6664
Epoch 4/50
10/10 [=======

In [41]:
plot_loss(history)

In [44]:
# load model
model = tf.keras.models.load_model('data/models/classification.keras', custom_objects={'abs': tf.math.abs})
#
prediction = model.predict(np.array(x_train))
print("prediction shape:", prediction.shape)

1458/1458 [==============================] - 7s 5ms/step
prediction shape: (46656, 2)


In [29]:
pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
pressures = np.load("data/initial kernels/Pressure_Silica.npy")
NX, NY = 4 , 5
figure, axis = plt.subplots(NX, NY)
for i in range(NX):
    for j in range(NY):
        k = np.random.randint(0, len(x_train))
        x_scale_factor = max(pore_widths)/len(x_train[k])
        axis[i, j].plot(pore_widths/x_scale_factor, prediction[k], marker=".", label=f"Prediction") 
        axis[i, j].plot(pore_widths/x_scale_factor, y_train[k], marker=".", label="Real distribution")
        axis[i, j].plot(x_train[k], label="Isotherm")
        ###
        ### Distribution and Isotherm 
        # axis[i, j].plot(pore_widths/x_scale_factor), y_train[k], marker=".", label="Distribution")
        # axis[i, j].plot(x_train[k], marker=".", label="Isotherm")
        ###
        
        axis[i, j].set_title(f"№ {k}")
        axis[i, j].title.set_size(10)
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plt.legend()
plt.show()

In [28]:
import generator
gen = generator.Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
                        path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
                        path_p_d="data/initial kernels/Pressure_Carbon.npy",
                        path_p_s="data/initial kernels/Pressure_Carbon.npy",
                        path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
                )
gen.generate_pore_distribution(d0_1=5.41454887, d0_2=16.07756851, sigma1=4.19612833, sigma2=6.50303474, a=0.66)
gen.calculate_calculate_isotherms_right()
test = gen.n_s[min_exp_pressure_i:max_exp_pressure_i]
test = test - min(test)
test /= max(test)
# plt.plot(test, marker=".")
prediction2 = model.predict(np.array([test])).T
plt.plot(pore_widths, prediction2, marker=".")
plt.plot(pore_widths, gen.pore_distribution, marker=".", label="real")
plt.legend()
plt.show()

1/1 [==============================] - 0s 121ms/step


ValueError: x and y must have same first dimension, but have shapes (128,) and (130,)

In [ ]:
gen.pore_distribution = prediction[i]
gen.calculate_calculate_isotherms_right()
plt.plot(gen.pressures_s[min_exp_pressure_i:max_exp_pressure_i], x_train[i], marker=".", label="Real")
plt.plot(gen.pressures_s[min_exp_pressure_i:max_exp_pressure_i], gen.n_s[min_exp_pressure_i:max_exp_pressure_i], marker=".", label="Net")
plt.legend()
plt.show()

In [ ]:
model.save('data/models/carbon_two_equal_peaks_conv.keras')